In [25]:
from tensorflow.keras.layers import  Input,Dense,Flatten,Conv2D,MaxPooling2D,Dropout,BatchNormalization,LeakyReLU
from tensorflow.keras.models import  Sequential
from tensorflow.keras.datasets import  fashion_mnist
import matplotlib.pyplot as  plt
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.optimizers import SGD,Adam
from keras_tuner.tuners import RandomSearch


In [26]:
# load the fashio mnist datasets
data = fashion_mnist.load_data()

In [27]:
# split the data into training and testing datasets
(train_images, train_labels), (test_images, test_labels) = data

In [28]:
#normalize the data
train_images = train_images / 255.0
test_images = test_images / 255.0


In [29]:
def create_model(hp):
    # define the hyperparameters
    activation = hp.Choice('activation', ['relu', 'leaky_relu'])
    optimizer = hp.Choice('optimizer', ['adam', 'sgd'])
    if optimizer =='sgd':
        learning_rate = hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')
        momentum = hp.Float('momentum', 0.0,0.5, 0.9)
        optimizer = SGD(learning_rate=learning_rate,momentum=momentum)
    if activation == 'leaky_relu':
        alpha = hp.Float('alpha', 0.01, 0.2)
        activation = LeakyReLU(alpha=alpha)
    model = Sequential([
        Input(shape=(28, 28, 1)),
        Conv2D(32, (3, 3), activation=activation, padding='same',name="conv1"),
        # MaxPooling2D((2, 2)),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation=activation, padding='same',name="conv2"),
        MaxPooling2D((2, 2)),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation=activation, padding='same',name="conv3"),
        MaxPooling2D((2, 2)),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation=activation, padding='same',name="conv4"),
        MaxPooling2D((2, 2)),
        BatchNormalization(),
        Flatten(),
        Dense(128, activation=activation),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

    return model

In [30]:
tuner = RandomSearch(
    create_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='fashion_mnist_tuning_2',
)

In [31]:
tuner.search(
    train_images,
    train_labels,
    epochs=5,
    validation_split=0.2,
    batch_size=32
)

Trial 5 Complete [00h 04m 40s]
val_accuracy: 0.9159999887148539

Best val_accuracy So Far: 0.9196111162503561
Total elapsed time: 00h 27m 01s


In [32]:
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters()[0]

In [33]:
best_hyperparameters.values

{'activation': 'leaky_relu',
 'optimizer': 'adam',
 'learning_rate': 0.0005264363579822869,
 'momentum': 0.0,
 'alpha': 0.16929982827406403}